In [1]:
import random
import pickle
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence
from keras import backend as K

Using TensorFlow backend.


In [9]:
# Here is the section dedicated to contatain the results of the sampling
# Purpose here is to try and use the UNET architecture and understand the
# first steps of building and training the neural network
sampled_pickles = '/home/omar/Desktop/sampled_output/'
smooth = 1

In [10]:
# First step is to choose random 26 patients as training pool. The
# remaining 3 are our test pool.

patients = os.listdir(sampled_pickles)
random.shuffle(patients)
training_patients = patients[:26]
test_patients = patients[26:29]

In [11]:
image_width = 256
image_height = 256
batch_size= 16

def prepare_data(path, list_of_patients):
    training_images = []
    training_masks = []
    
    for patient in list_of_patients:
        print('Current Patient: %s' % (patient))
        patient_path = path + patient
        pickle_file = open("%s"%(patient_path), "rb")
        patient_data = pickle.load(pickle_file)
        for scan in range(len(patient_data)):
            if scan != 4:
                for array in range(len(patient_data[scan])):
                    training_images.append(patient_data[scan][array])
                    training_masks.append(patient_data[4][scan][array])
        print("IMAGES: ", len(training_images))
        print("MASKS: ", len(training_masks))
        return training_images, training_masks

In [12]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def UNet():
    inputs = Input((image_width, image_height, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model

In [15]:
print('-'*30)
print('Generating Data and masks')
print('-'*30)
imgs, masks = prepare_data(sampled_pickles, training_patients)
print('-'*30)
print('Compiling and creating model')
print('-'*30)
model = UNet()
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
print('-'*30)
print('Fitting model...')
print('-'*30)
model.fit(imgs, masks, batch_size=32, nb_epoch=20, verbose=1,
          shuffle=True, validation_split=0.2, callbacks=[model_checkpoint])
print('-'*30)
print('Fitting Complete')
print('-'*30)

------------------------------
Generating Data and masks
------------------------------
Current Patient: TCGA-06-6389.pickle
IMAGES:  201
MASKS:  201
------------------------------
Compiling and creating model
------------------------------
------------------------------
Fitting model...
------------------------------


/home/omar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 201 arrays: [array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0,...